In [161]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

In [162]:
canada_rate_file = "Resources/Exchangerates_Canada.csv"
china_rate_file = "Resources/Exchangerates_China.csv"
euro_rate_file = "Resources/Exchangerates_Euro.csv"
hongkong_rate_file = "Resources/Exchangerates_HongKong.csv"
india_rate_file = "Resources/Exchangerates_India.csv"
japan_rate_file = "Resources/Exchangerates_Japan.csv"
korea_rate_file = "Resources/Exchangerates_Korea.csv"
southafrica_rate_file = "Resources/Exchangerates_SouthAfrica.csv"
swiss_rate_file = "Resources/Exchangerates_Swiss.csv"

index_data_file = "Resources/indexData.csv"
info_data_file = "Resources/infoData.csv"

In [163]:
canada_df = pd.read_csv(canada_rate_file)
china_df = pd.read_csv(china_rate_file)
euro_df = pd.read_csv(euro_rate_file)
hongkong_df = pd.read_csv(hongkong_rate_file)
india_df = pd.read_csv(india_rate_file)
japan_df = pd.read_csv(japan_rate_file)
korea_df = pd.read_csv(korea_rate_file)
southafrica_df = pd.read_csv(southafrica_rate_file)
swiss_df = pd.read_csv(swiss_rate_file)

data_df = pd.read_csv(index_data_file)
info_df = pd.read_csv(info_data_file)

In [164]:
info_df

,Region,Exchange,Index,Currency
0,United States,New York Stock Exchange,NYA,USD
1,United States,NASDAQ,IXIC,USD
2,Hong Kong,Hong Kong Stock Exchange,HSI,HKD
3,China,Shanghai Stock Exchange,000001.SS,CNY
4,Japan,Tokyo Stock Exchange,N225,JPY
5,Europe,Euronext,N100,EUR
6,China,Shenzhen Stock Exchange,399001.SZ,CNY
7,Canada,Toronto Stock Exchange,GSPTSE,CAD
8,India,National Stock Exchange of India,NSEI,INR
9,Germany,Frankfurt Stock Exchange,GDAXI,EUR


In [165]:
data_df

,Index,Date,Open,High,Low,Close,Adj Close,Volume
0,NYA,1965-12-31,528.690002,528.690002,528.690002,528.690002,528.690002,0.0
1,NYA,1966-01-03,527.210022,527.210022,527.210022,527.210022,527.210022,0.0
2,NYA,1966-01-04,527.840027,527.840027,527.840027,527.840027,527.840027,0.0
3,NYA,1966-01-05,531.119995,531.119995,531.119995,531.119995,531.119995,0.0
4,NYA,1966-01-06,532.070007,532.070007,532.070007,532.070007,532.070007,0.0
...,...,...,...,...,...,...,...,...
112452,N100,2021-05-27,1241.119995,1251.910034,1241.119995,1247.069946,1247.069946,379696400.0
112453,N100,2021-05-28,1249.469971,1259.209961,1249.030029,1256.599976,1256.599976,160773400.0
112454,N100,2021-05-31,1256.079956,1258.880005,1248.140015,1248.930054,1248.930054,91173700.0
112455,N100,2021-06-01,1254.609985,1265.660034,1254.609985,1258.579956,1258.579956,155179900.0


In [166]:
# keep only stock exchange data from year 2000 and later
data_df = data_df.loc[data_df['Date']>= '2000-01-01']

#remove taiwan data as we do not have exchange rates for this stock exchange.
data_df = data_df.loc[data_df['Index']!= 'TWII']

#drop unecessary column
data_df.drop(columns='Adj Close',inplace= True)

# stock exchange data with null values
len(data_df)

65121

In [167]:
data_df.dropna(inplace=True)
len(data_df)

64281

In [168]:
#convert date column string from exchnage rate data frames into date objects to macth data_df

canada_df['Date']= pd.to_datetime(canada_df['Date'])
china_df['Date']= pd.to_datetime(china_df['Date'])
euro_df['Date']= pd.to_datetime(euro_df['Date'])
hongkong_df['Date']= pd.to_datetime(hongkong_df['Date'])
india_df['Date']= pd.to_datetime(india_df['Date'])
japan_df['Date']= pd.to_datetime(japan_df['Date'])
korea_df['Date']= pd.to_datetime(korea_df['Date'])
southafrica_df['Date']= pd.to_datetime(southafrica_df['Date'])
swiss_df['Date']= pd.to_datetime(swiss_df['Date'])

data_df['Date']= pd.to_datetime(data_df['Date'])

In [169]:
#add column for currency type to merge on later
canada_df['Currency']= 'CAD'
china_df['Currency']= 'CNY'
euro_df['Currency']= 'EUR'
hongkong_df['Currency']= 'HKD'
india_df['Currency']= 'INR'
japan_df['Currency']= 'JPY'
korea_df['Currency']= 'KRW'
southafrica_df['Currency']= 'ZAR'
swiss_df['Currency']= 'CHF'

In [170]:
#rename column
canada_df.rename(columns = {'CAD':'exchange_rate'}, inplace = True)
china_df.rename(columns = {'CNY':'exchange_rate'}, inplace = True)
euro_df.rename(columns = {'EUR':'exchange_rate'}, inplace = True)
hongkong_df.rename(columns = {'HKD':'exchange_rate'}, inplace = True)
india_df.rename(columns = {'INR':'exchange_rate'}, inplace = True)
japan_df.rename(columns = {'JPY':'exchange_rate'}, inplace = True)
korea_df.rename(columns = {'KRW':'exchange_rate'}, inplace = True)
southafrica_df.rename(columns = {'ZAR':'exchange_rate'}, inplace = True)
swiss_df.rename(columns = {'CHF':'exchange_rate'}, inplace = True)

In [171]:
#concatenate all exchange rate data frames into one
exchange_rate_df = pd.concat([canada_df, china_df, euro_df, hongkong_df, india_df, japan_df, korea_df, southafrica_df, swiss_df])
exchange_rate_df

,Date,USD,exchange_rate,Currency
0,2021-09-02,1,1.2615,CAD
1,2021-09-01,1,1.2591,CAD
2,2021-08-31,1,1.2587,CAD
3,2021-08-30,1,1.2598,CAD
4,2021-08-29,1,1.2681,CAD
...,...,...,...,...
8273,1999-01-08,1,1.3842,CHF
8274,1999-01-07,1,1.3897,CHF
8275,1999-01-06,1,1.3724,CHF
8276,1999-01-05,1,1.3675,CHF


In [172]:
exchange_rate_df.dtypes

Date             datetime64[ns]
USD                       int64
exchange_rate            object
Currency                 object
dtype: object

In [173]:
exchange_rate_df['exchange_rate'] = exchange_rate_df['exchange_rate'].replace(',', '',regex=True)

In [174]:
exchange_rate_df['exchange_rate'] = exchange_rate_df['exchange_rate'].astype(float)
exchange_rate_df

,Date,USD,exchange_rate,Currency
0,2021-09-02,1,1.2615,CAD
1,2021-09-01,1,1.2591,CAD
2,2021-08-31,1,1.2587,CAD
3,2021-08-30,1,1.2598,CAD
4,2021-08-29,1,1.2681,CAD
...,...,...,...,...
8273,1999-01-08,1,1.3842,CHF
8274,1999-01-07,1,1.3897,CHF
8275,1999-01-06,1,1.3724,CHF
8276,1999-01-05,1,1.3675,CHF


In [175]:
# keep only exchange rate data from year 2000 and later
exchange_rate_df = exchange_rate_df.loc[exchange_rate_df['Date']>= '2000-01-01']


exchange_rate_df

,Date,USD,exchange_rate,Currency
0,2021-09-02,1,1.2615,CAD
1,2021-09-01,1,1.2591,CAD
2,2021-08-31,1,1.2587,CAD
3,2021-08-30,1,1.2598,CAD
4,2021-08-29,1,1.2681,CAD
...,...,...,...,...
7911,2000-01-05,1,1.5490,CHF
7912,2000-01-04,1,1.5578,CHF
7913,2000-01-03,1,1.5900,CHF
7914,2000-01-02,1,1.5978,CHF


In [176]:
# merge info df into data df
merge_df = pd.merge(data_df,info_df,on='Index',how='left')
merge_df

,Index,Date,Open,High,Low,Close,Volume,Region,Exchange,Currency
0,NYA,2000-01-03,6762.109863,6762.109863,6762.109863,6762.109863,0.0,United States,New York Stock Exchange,USD
1,NYA,2000-01-04,6543.759766,6543.759766,6543.759766,6543.759766,0.0,United States,New York Stock Exchange,USD
2,NYA,2000-01-05,6567.029785,6567.029785,6567.029785,6567.029785,0.0,United States,New York Stock Exchange,USD
3,NYA,2000-01-06,6635.439941,6635.439941,6635.439941,6635.439941,0.0,United States,New York Stock Exchange,USD
4,NYA,2000-01-07,6792.669922,6792.669922,6792.669922,6792.669922,0.0,United States,New York Stock Exchange,USD
...,...,...,...,...,...,...,...,...,...,...
64276,N100,2021-05-27,1241.119995,1251.910034,1241.119995,1247.069946,379696400.0,Europe,Euronext,EUR
64277,N100,2021-05-28,1249.469971,1259.209961,1249.030029,1256.599976,160773400.0,Europe,Euronext,EUR
64278,N100,2021-05-31,1256.079956,1258.880005,1248.140015,1248.930054,91173700.0,Europe,Euronext,EUR
64279,N100,2021-06-01,1254.609985,1265.660034,1254.609985,1258.579956,155179900.0,Europe,Euronext,EUR


In [177]:
stock_exchange_df = pd.merge(merge_df,exchange_rate_df, on=['Currency','Date'], how='left')
stock_exchange_df

,Index,Date,Open,High,Low,Close,Volume,Region,Exchange,Currency,USD,exchange_rate
0,NYA,2000-01-03,6762.109863,6762.109863,6762.109863,6762.109863,0.0,United States,New York Stock Exchange,USD,NaN,NaN
1,NYA,2000-01-04,6543.759766,6543.759766,6543.759766,6543.759766,0.0,United States,New York Stock Exchange,USD,NaN,NaN
2,NYA,2000-01-05,6567.029785,6567.029785,6567.029785,6567.029785,0.0,United States,New York Stock Exchange,USD,NaN,NaN
3,NYA,2000-01-06,6635.439941,6635.439941,6635.439941,6635.439941,0.0,United States,New York Stock Exchange,USD,NaN,NaN
4,NYA,2000-01-07,6792.669922,6792.669922,6792.669922,6792.669922,0.0,United States,New York Stock Exchange,USD,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
64276,N100,2021-05-27,1241.119995,1251.910034,1241.119995,1247.069946,379696400.0,Europe,Euronext,EUR,1.0,0.8198
64277,N100,2021-05-28,1249.469971,1259.209961,1249.030029,1256.599976,160773400.0,Europe,Euronext,EUR,1.0,0.8236
64278,N100,2021-05-31,1256.079956,1258.880005,1248.140015,1248.930054,91173700.0,Europe,Euronext,EUR,1.0,0.8196
64279,N100,2021-06-01,1254.609985,1265.660034,1254.609985,1258.579956,155179900.0,Europe,Euronext,EUR,1.0,0.8180


In [178]:
#check for null values
df_null = stock_exchange_df.loc[stock_exchange_df['exchange_rate'].isnull()]
df_null['Index'].unique()

array(['NYA', 'IXIC'], dtype=object)

In [179]:
#There is no exchange rate values for USD sice that is our base currency, solution: fill all null values with 1.
stock_exchange_df['exchange_rate'].fillna(1, inplace = True)
stock_exchange_df['USD'].fillna(1, inplace = True)
stock_exchange_df

,Index,Date,Open,High,Low,Close,Volume,Region,Exchange,Currency,USD,exchange_rate
0,NYA,2000-01-03,6762.109863,6762.109863,6762.109863,6762.109863,0.0,United States,New York Stock Exchange,USD,1.0,1.0000
1,NYA,2000-01-04,6543.759766,6543.759766,6543.759766,6543.759766,0.0,United States,New York Stock Exchange,USD,1.0,1.0000
2,NYA,2000-01-05,6567.029785,6567.029785,6567.029785,6567.029785,0.0,United States,New York Stock Exchange,USD,1.0,1.0000
3,NYA,2000-01-06,6635.439941,6635.439941,6635.439941,6635.439941,0.0,United States,New York Stock Exchange,USD,1.0,1.0000
4,NYA,2000-01-07,6792.669922,6792.669922,6792.669922,6792.669922,0.0,United States,New York Stock Exchange,USD,1.0,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
64276,N100,2021-05-27,1241.119995,1251.910034,1241.119995,1247.069946,379696400.0,Europe,Euronext,EUR,1.0,0.8198
64277,N100,2021-05-28,1249.469971,1259.209961,1249.030029,1256.599976,160773400.0,Europe,Euronext,EUR,1.0,0.8236
64278,N100,2021-05-31,1256.079956,1258.880005,1248.140015,1248.930054,91173700.0,Europe,Euronext,EUR,1.0,0.8196
64279,N100,2021-06-01,1254.609985,1265.660034,1254.609985,1258.579956,155179900.0,Europe,Euronext,EUR,1.0,0.8180


In [180]:
stock_exchange = stock_exchange_df['exchange_rate'].replace(0.0000,1, inplace=True)

In [181]:
#Do conversion of columns to USD
stock_exchange_df['Open_USD']=stock_exchange_df['Open']/stock_exchange_df['exchange_rate']
stock_exchange_df['High_USD']=stock_exchange_df['High']/stock_exchange_df['exchange_rate']
stock_exchange_df['Low_USD']=stock_exchange_df['Low']/stock_exchange_df['exchange_rate']
stock_exchange_df['Close_USD']=stock_exchange_df['Close']/stock_exchange_df['exchange_rate']
stock_exchange_df

,Index,Date,Open,High,Low,Close,Volume,Region,Exchange,Currency,USD,exchange_rate,Open_USD,High_USD,Low_USD,Close_USD
0,NYA,2000-01-03,6762.109863,6762.109863,6762.109863,6762.109863,0.0,United States,New York Stock Exchange,USD,1.0,1.0000,6762.109863,6762.109863,6762.109863,6762.109863
1,NYA,2000-01-04,6543.759766,6543.759766,6543.759766,6543.759766,0.0,United States,New York Stock Exchange,USD,1.0,1.0000,6543.759766,6543.759766,6543.759766,6543.759766
2,NYA,2000-01-05,6567.029785,6567.029785,6567.029785,6567.029785,0.0,United States,New York Stock Exchange,USD,1.0,1.0000,6567.029785,6567.029785,6567.029785,6567.029785
3,NYA,2000-01-06,6635.439941,6635.439941,6635.439941,6635.439941,0.0,United States,New York Stock Exchange,USD,1.0,1.0000,6635.439941,6635.439941,6635.439941,6635.439941
4,NYA,2000-01-07,6792.669922,6792.669922,6792.669922,6792.669922,0.0,United States,New York Stock Exchange,USD,1.0,1.0000,6792.669922,6792.669922,6792.669922,6792.669922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64276,N100,2021-05-27,1241.119995,1251.910034,1241.119995,1247.069946,379696400.0,Europe,Euronext,EUR,1.0,0.8198,1513.930221,1527.092015,1513.930221,1521.188029
64277,N100,2021-05-28,1249.469971,1259.209961,1249.030029,1256.599976,160773400.0,Europe,Euronext,EUR,1.0,0.8236,1517.083500,1528.909618,1516.549331,1525.740622
64278,N100,2021-05-31,1256.079956,1258.880005,1248.140015,1248.930054,91173700.0,Europe,Euronext,EUR,1.0,0.8196,1532.552411,1535.968771,1522.864830,1523.828763
64279,N100,2021-06-01,1254.609985,1265.660034,1254.609985,1258.579956,155179900.0,Europe,Euronext,EUR,1.0,0.8180,1533.753038,1547.261655,1533.753038,1538.606303


In [182]:
stock_exchange_df.loc[stock_exchange_df['Exchange']== 'Shenzhen Stock Exchange']

,Index,Date,Open,High,Low,Close,Volume,Region,Exchange,Currency,USD,exchange_rate,Open_USD,High_USD,Low_USD,Close_USD
26603,399001.SZ,2000-01-04,3374.110107,3512.300049,3360.209961,3497.060059,0.0,China,Shenzhen Stock Exchange,CNY,1.0,1.0000,3374.110107,3512.300049,3360.209961,3497.060059
26604,399001.SZ,2000-01-05,3500.129883,3589.179932,3468.689941,3486.290039,0.0,China,Shenzhen Stock Exchange,CNY,1.0,1.0000,3500.129883,3589.179932,3468.689941,3486.290039
26605,399001.SZ,2000-01-06,3475.459961,3663.219971,3454.350098,3655.199951,0.0,China,Shenzhen Stock Exchange,CNY,1.0,1.0000,3475.459961,3663.219971,3454.350098,3655.199951
26606,399001.SZ,2000-01-07,3701.479980,3848.060059,3701.479980,3828.040039,0.0,China,Shenzhen Stock Exchange,CNY,1.0,1.0000,3701.479980,3848.060059,3701.479980,3828.040039
26607,399001.SZ,2000-01-10,3881.750000,3929.060059,3832.199951,3921.479980,0.0,China,Shenzhen Stock Exchange,CNY,1.0,1.0000,3881.750000,3929.060059,3832.199951,3921.479980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31782,399001.SZ,2021-05-27,14796.049810,14955.299810,14720.669920,14897.190430,1502100.0,China,Shenzhen Stock Exchange,CNY,1.0,6.3788,2319.566346,2344.531857,2307.749094,2335.422090
31783,399001.SZ,2021-05-28,14899.330080,14977.250000,14778.820310,14852.879880,1651600.0,China,Shenzhen Stock Exchange,CNY,1.0,6.3764,2336.636673,2348.856722,2317.737330,2329.351967
31784,399001.SZ,2021-05-31,14874.889650,14996.379880,14836.150390,14996.379880,1460400.0,China,Shenzhen Stock Exchange,CNY,1.0,6.3701,2335.110854,2354.182804,2329.029433,2354.182804
31785,399001.SZ,2021-06-01,14963.950200,15039.019530,14820.740230,15034.780270,1515600.0,China,Shenzhen Stock Exchange,CNY,1.0,6.3839,2344.013879,2355.773043,2321.580888,2355.108988


In [183]:
stock_exchange_df['Exchange'].unique()

array(['New York Stock Exchange', 'NASDAQ', 'Hong Kong Stock Exchange',
       'Shanghai Stock Exchange', 'Toronto Stock Exchange',
       'Shenzhen Stock Exchange', 'National Stock Exchange of India',
       'Frankfurt Stock Exchange', 'Korea Exchange', 'SIX Swiss Exchange',
       'Johannesburg Stock Exchange', 'Tokyo Stock Exchange', 'Euronext'],
      dtype=object)

In [184]:
stock_exchange_df= stock_exchange_df.rename(columns={'Index':'Stock_index'})
stock_exchange_df

,Stock_index,Date,Open,High,Low,Close,Volume,Region,Exchange,Currency,USD,exchange_rate,Open_USD,High_USD,Low_USD,Close_USD
0,NYA,2000-01-03,6762.109863,6762.109863,6762.109863,6762.109863,0.0,United States,New York Stock Exchange,USD,1.0,1.0000,6762.109863,6762.109863,6762.109863,6762.109863
1,NYA,2000-01-04,6543.759766,6543.759766,6543.759766,6543.759766,0.0,United States,New York Stock Exchange,USD,1.0,1.0000,6543.759766,6543.759766,6543.759766,6543.759766
2,NYA,2000-01-05,6567.029785,6567.029785,6567.029785,6567.029785,0.0,United States,New York Stock Exchange,USD,1.0,1.0000,6567.029785,6567.029785,6567.029785,6567.029785
3,NYA,2000-01-06,6635.439941,6635.439941,6635.439941,6635.439941,0.0,United States,New York Stock Exchange,USD,1.0,1.0000,6635.439941,6635.439941,6635.439941,6635.439941
4,NYA,2000-01-07,6792.669922,6792.669922,6792.669922,6792.669922,0.0,United States,New York Stock Exchange,USD,1.0,1.0000,6792.669922,6792.669922,6792.669922,6792.669922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64276,N100,2021-05-27,1241.119995,1251.910034,1241.119995,1247.069946,379696400.0,Europe,Euronext,EUR,1.0,0.8198,1513.930221,1527.092015,1513.930221,1521.188029
64277,N100,2021-05-28,1249.469971,1259.209961,1249.030029,1256.599976,160773400.0,Europe,Euronext,EUR,1.0,0.8236,1517.083500,1528.909618,1516.549331,1525.740622
64278,N100,2021-05-31,1256.079956,1258.880005,1248.140015,1248.930054,91173700.0,Europe,Euronext,EUR,1.0,0.8196,1532.552411,1535.968771,1522.864830,1523.828763
64279,N100,2021-06-01,1254.609985,1265.660034,1254.609985,1258.579956,155179900.0,Europe,Euronext,EUR,1.0,0.8180,1533.753038,1547.261655,1533.753038,1538.606303


In [185]:
engine = create_engine('sqlite:///stock_exchange_db.sqlite', echo=True)
sqlite_connection = engine.connect()

In [186]:
sqlite_table = "stock_exchange_2"
stock_exchange_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace')

2021-10-05 19:38:08,735 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("stock_exchange_2")
2021-10-05 19:38:08,737 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-05 19:38:08,744 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("stock_exchange_2")
2021-10-05 19:38:08,746 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-05 19:38:08,752 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-10-05 19:38:08,754 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-05 19:38:08,761 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("stock_exchange_2")
2021-10-05 19:38:08,764 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-05 19:38:08,775 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-10-05 19:38:08,780 INFO sqlalchemy.engine.Engine [raw sql] ('stock_exchange_2',)
2021-10-05 19:38:08,789 INFO sqlalchemy.engine.Engine PR

In [187]:
sqlite_connection.close()

In [188]:
import os
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base

In [189]:
engine = create_engine('sqlite:///stock_exchange_db.sqlite', echo=True)

In [190]:
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

2021-10-05 19:38:17,529 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-10-05 19:38:17,537 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-05 19:38:17,546 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("stock_exchange")
2021-10-05 19:38:17,558 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-05 19:38:17,574 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-10-05 19:38:17,578 INFO sqlalchemy.engine.Engine [raw sql] ('stock_exchange',)
2021-10-05 19:38:17,586 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("stock_exchange")
2021-10-05 19:38:17,590 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-05 19:38:17,595 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("stock_exchange")
2021-10-05 19:38:17,598 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-05 19:38:17,603 INFO sqlalchemy.engine.Engin

In [191]:
print(Base.classes.keys())

['stock_exchange']
